In [19]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [20]:
LOCAL_PATH = 'data'
RAW_TRACE = 'Skype_HongKong.pcapng'
TCP_TRACE_FEATURE_FILE = 'Skype_HongKong_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'Skype_HongKong_udp_pkt.csv'
TCP_FLOW_FEATURE_FILE = 'Skype_HongKong_tcp_flow.csv'
UDP_FLOW_FEATURE_FILE = 'Skype_HongKong_udp_flow.csv'
# BUCKET_NAME = '' # replace with your bucket name
# KEY = '' # replace with your object key

In [21]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [25]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 5.5 s, sys: 336 ms, total: 5.84 s
Wall time: 35.7 s
Conversion done
CPU times: user 396 ms, sys: 73.4 ms, total: 470 ms
Wall time: 30.6 s


In [26]:
def to_csv(trace_df, extract_func, file):
    extract_func(trace_df, 1.0, upsampled=True).to_csv(file, index=False)

In [27]:
from python import flow_feature
reload(flow_feature)

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE))
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE))
    else:
        raise


1.0 samp rate with upsampling:  55%|█████▌    | 395/717 [00:12<00:10, 29.95it/s]


1.0 samp rate with upsampling: 100%|██████████| 717/717 [00:19<00:00, 50.07it/s]

1.0 samp rate with upsampling:  72%|███████▏  | 151/211 [00:07<00:02, 22.98it/s]


1.0 samp rate with upsampling: 100%|██████████| 211/211 [00:10<00:00, 18.06it/s]
